In [62]:
from ortools.sat.python import cp_model
import pandas as pd

In [157]:
# Read the requests tables
df = pd.read_table('requests.tsv', sep='\t', header=None)

# Change the scale. 1 -> 6, 2 -> 5, etc.
df = (6 - df).fillna(0)

In [158]:

# This program tries to find an optimal assignment of kids to shifts
# subject to some constraints (see below).
# The optimal assignment maximizes the number of fulfilled shift requests.
num_kids = 29
num_kids_per_day = num_kids - 24
num_days = 7
all_kids = range(num_kids)
all_days = range(num_days)
shift_requests = df.values
fixed_assignments = [(26, 1, 1), (3, 1, 1), (3, 0, 1), (21, 0, 1), (25, 0, 1)] + [(n, 0, 0) for n in all_kids if n not in (3, 21, 25)]
# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# shifts[(n, d)]: kid 'n' gets day 'd'.
shifts = {}
for n in all_kids:
    for d in all_days:
        shifts[(n, d)] = model.NewBoolVar('shift_n%id%id' % (n, d))

# Each day has exactly num_kids_per_day kids
for d in all_days[1:]:        
    model.Add(sum(shifts[(n, d)] for n in all_kids) == num_kids_per_day)
model.Add(sum(shifts[(n, 0)] for n in all_kids) == 3)

    
# Fixed assignments.
for n, d, r in fixed_assignments:
    model.Add(shifts[(n, d)] == r)

# Twins
for d in all_days:
    model.Add(shifts[(0, d)] == shifts[(1, d)])
    
# Try to distribute the shifts evenly, so that each nurse works
# min_shifts_per_nurse shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of nurses, some nurses will
# be assigned one more shift.
min_shifts_per_kid = (num_kids_per_day * num_days) // num_kids
if num_kids_per_day * num_days % num_kids == 0:
    max_shifts_per_kid = min_shifts_per_kid
else:
    max_shifts_per_kid = min_shifts_per_kid + 1
for n in all_kids:
    num_shifts_worked = 0
    for d in all_days:
        num_shifts_worked += shifts[(n, d)]
    model.Add(min_shifts_per_kid <= num_shifts_worked)
    model.Add(num_shifts_worked <= max_shifts_per_kid)

# pylint: disable=g-complex-comprehension
model.Maximize(sum(int(shift_requests[n][d]) * shifts[(n, d)] for n in all_kids for d in all_days))
# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.Solve(model)
for d in all_days:
    print('Day', d)
    for n in all_kids:
        if solver.Value(shifts[(n, d)]) == 1:
            print('Kid', n, '. Requests = ', shift_requests[n][d])            
    print()

# Statistics.
print()
print('Statistics')
print('  - Number of shift requests met = %i' % solver.ObjectiveValue(),
      '(out of', num_kids * min_shifts_per_kid, ')')
print('  - wall time       : %f s' % solver.WallTime())


Day 0
Kid 3 . Requests =  5.0
Kid 21 . Requests =  5.0
Kid 25 . Requests =  5.0

Day 1
Kid 2 . Requests =  5.0
Kid 3 . Requests =  5.0
Kid 8 . Requests =  5.0
Kid 13 . Requests =  5.0
Kid 26 . Requests =  5.0

Day 2
Kid 6 . Requests =  4.0
Kid 10 . Requests =  5.0
Kid 16 . Requests =  5.0
Kid 22 . Requests =  5.0
Kid 26 . Requests =  5.0

Day 3
Kid 10 . Requests =  5.0
Kid 12 . Requests =  5.0
Kid 15 . Requests =  4.0
Kid 16 . Requests =  5.0
Kid 27 . Requests =  5.0

Day 4
Kid 0 . Requests =  5.0
Kid 1 . Requests =  5.0
Kid 5 . Requests =  5.0
Kid 7 . Requests =  5.0
Kid 14 . Requests =  4.0

Day 5
Kid 9 . Requests =  5.0
Kid 11 . Requests =  4.0
Kid 19 . Requests =  5.0
Kid 23 . Requests =  5.0
Kid 24 . Requests =  5.0

Day 6
Kid 4 . Requests =  4.0
Kid 17 . Requests =  5.0
Kid 18 . Requests =  5.0
Kid 20 . Requests =  5.0
Kid 28 . Requests =  5.0


Statistics
  - Number of shift requests met = 160 (out of 29 )
  - wall time       : 0.043349 s


In [159]:
res = pd.DataFrame([[solver.Value(shifts[(n, d)]) for n in all_kids] for d in all_days]).T

In [162]:
res.to_csv('res.tsv', sep='\t')